In [ ]:
!pip install faker pymongo

In [ ]:
from pymongo import MongoClient # pip install pymongo
from faker import Faker # pip install faker

from utils.config import load_config

In [ ]:
config = load_config("config.yaml")
config

In [ ]:
f = Faker("pl_PL")
dane = []
for _ in range(100):
    dane.append(
        {
            "imie": f.first_name(),
            "nazwisko": f.last_name(),
            "adres": f.address(),
            "email": f.email(),
            "data_urodzenia": f.date_of_birth(
                minimum_age=18, maximum_age=90
            ).isoformat(),
        }
    )

In [ ]:
dane

In [ ]:
# Połącz z lokalną bazą MongoDB
client = MongoClient(
    host=config["mongo_db"]["host"],
    port=config["mongo_db"]["port"],
    username=config["mongo_db"]["user"],
    password=config["mongo_db"]["password"],
)

# Wybierz bazę danych
db = client[config["mongo_db"]["database"]]

# Wybierz kolekcję
collection = db[config["mongo_db"]["collection"]]


In [ ]:
client

In [ ]:
for record in dane:
    collection.insert_one(record)

In [ ]:
# Przykładowe pobranie dokumentów
for doc in collection.find():
    print(doc)

In [ ]:
# wyszukiwanie dokumentów z nazwiskiem "Kowalski"
for doc in collection.find({"nazwisko": "Kowalski"}):
    print(doc)

In [ ]:
# agregacja - zliczanie wystąpień imion
pipeline = [
    {"$group": {"_id": "$imie", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
]

wyniki = list(collection.aggregate(pipeline))
for wynik in wyniki:
    print(f"{wynik['_id']}: {wynik['count']}")